## Evaluation Retrieval (w/o RAG)

In [ ]:
import pandas as pd

testset = pd.read_csv('document/testset.csv')
# testset = testset.drop(columns=['Unnamed: 0'])
testset.head(1)

In [ ]:
# import openai

# openai.api_key = ""

# def ask_chatgpt(prompt):
    
#     completion = openai.ChatCompletion.create(
#       model="gpt-3.5-turbo",
#       temperature=0,
#       messages=[
#             {"role": "system", "content": "You are an artificial intelligence assistant and National Taiwan University campus guide"},
#             {"role": "user", "content": prompt},
#         ]
#     )
#     return completion

In [ ]:
# from transformers import AutoTokenizer
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# model_id = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# llm = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     # attn_implementation="flash_attention_2" # optional
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id)


# def generate_text(prompt_text):

#     messages = [
#         {
#             "role": "system",
#             "content": "你是一個人工智慧助理以及台灣大學校園導覽員",
#         },
#         {"role": "user", "content": prompt_text},
#     ]
#     prompt = tokenizer.apply_chat_template(messages)
#     print(f'Prompt:{prompt}')
#     outputs = llm(prompt)
#     return outputs

In [ ]:
# from langchain_community.llms import VLLM
# import torch
# # from transformers import pipeline
# from transformers import AutoTokenizer


# model_id = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# llm = VLLM(
#     model=model_id,
#     gpu_memory_utilization=0.95,
#     max_new_tokens=512,
#     torch_dtype=torch.bfloat16,
#     enforce_eager=True,
#     max_seq_len=14000,
#     trust_remote_code=True,  # mandatory for hf models
#     max_model_len=14000,
#     # attn_implementation="flas
#     # h_attention_2", # optional
#     # top_k=50,
#     # top_p=0.95,
#     # temperature=0.7,
#     # gpu_memory_utilization=0.8
# )


# def generate_text(prompt_text):

#     # pipe = pipeline("text-generation", device=0)
#     tokenizer = AutoTokenizer.from_pretrained(model_id)

#     # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
#     messages = [
#         {
#             "role": "system",
#             "content": "你是一個人工智慧助理以及台灣大學校園導覽員",
#         },
#         {"role": "user", "content": prompt_text},
#     ]
#     prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     print(f'Prompt:{prompt}')
#     outputs = llm(prompt)

#     # print(outputs[0]["generated_text"])
#     return outputs

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_id = "MediaTek-Research/Breeze-7B-Instruct-v0_1"
# model_id = "yentinglin/Taiwan-LLM-13B-v2.0-chat"
model_id = "taide/TAIDE-LX-7B-Chat"

# llm = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
    # torch_dtype=torch.bfloat16,
#     # attn_implementation="flash_attention_2" # optional
# ).cuda()
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    # torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    device_map="auto",
    # attn_implementation="flash_attention_2" # optional
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


def generate_text(prompt_text):

    messages = [
        {
            "role": "system",
            "content": "你是一個人工智慧助理以及台灣大學校園導覽員",
        },
        {"role": "user", "content": prompt_text},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    # print(f'Prompt:{prompt}')

    inputs = tokenizer(prompt, return_tensors="pt").to(0)

    outputs = llm.generate(**inputs, max_new_tokens=512)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [ ]:
# from transformers import AutoTokenizer
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# model_id = "MediaTek-Research/Breeze-7B-Instruct-v0_1"

# # llm = AutoModelForCausalLM.from_pretrained(
# #     model_id,
# #     device_map="auto",
#     # torch_dtype=torch.bfloat16,
# #     # attn_implementation="flash_attention_2" # optional
# # ).cuda()
# llm = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.bfloat16,
#     # load_in_8bit=True,
#     # attn_implementation="flash_attention_2" # optional
# ).cuda()
# tokenizer = AutoTokenizer.from_pretrained(model_id)


# def generate_text(prompt_text):

#     messages = [
#         {
#             "role": "system",
#             "content": "你是一個人工智慧助理以及台灣大學校園導覽員",
#         },
#         {"role": "user", "content": prompt_text},
#     ]
#     prompt = tokenizer.apply_chat_template(messages, tokenize=False)
#     print(f'Prompt:{prompt}')

#     inputs = tokenizer(prompt, return_tensors="pt").to(0)

#     outputs = llm.generate(**inputs, max_new_tokens=512)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)

#     return response


In [ ]:
from tqdm import tqdm
import time

responses = []
for query in tqdm(testset['Question']):
    # print(query)
    # try:
        # response = ask_chatgpt(query)['choices'][0]['message']['content']
        # response = generate_text(query)
    # except:
        # time.sleep(5)
        # response = ask_chatgpt(query)['choices'][0]['message']['content']
    response = generate_text(query)
    print(response)
    
    responses.append(response)
    # break

In [ ]:
testset[f'Answer_{model_id.split("/")[-1]}'] = responses
testset.to_csv(f'evaluation/testset_{model_id.split("/")[-1]}.csv', index=False)

## Evaluate with RAGAS

In [ ]:
import pandas as pd

testset = pd.read_csv('document/testset.csv')
testset.head(1)

In [ ]:
web_page = pd.read_csv('document/clean_content_4.csv')

In [ ]:
# import openai

# openai.api_key = ""

# def ask_chatgpt(prompt):
    
#     completion = openai.ChatCompletion.create(
#       model="gpt-3.5-turbo",
#       temperature=0,
#       messages=[
#             {"role": "system", "content": "You are an artificial intelligence assistant and National Taiwan University campus guide"},
#             {"role": "user", "content": prompt},
#         ]
#     )
#     return completion

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM


# llm = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto").eval()
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# def generate_text(prompt_text):

#     inputs = tokenizer(prompt_text, return_tensors="pt").to('cuda')

#     outputs = llm.generate(inputs["input_ids"], pad_token_id=50256, max_new_tokens=512)
#     response = tokenizer.decode(outputs[0].tolist(), skip_special_tokens=True)
        
#     return response

In [ ]:
# # pip install transformers>=4.34
# # pip install accelerate

# import torch
# from transformers import pipeline


# model_id = "yentinglin/Taiwan-LLM-7B-v2.1-chat"

# def generate_text(prompt_text):

#     pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

#     # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
#     messages = [
#         {
#             "role": "system",
#             "content": "你是一個工智慧助理以及台灣大學校園導覽員",
#         },
#         {"role": "user", "content": prompt_text},
#     ]
#     prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
#     # print(outputs[0]["generated_text"])
#     return outputs[0]["generated_text"]


In [ ]:
# from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
# from langchain_community.llms import VLLM
# import torch
# from transformers import pipeline

# model_id = "yentinglin/Taiwan-LLM-7B-v2.1-chat"

# llm = VLLM(
#     model=model_id,
#     # trust_remote_code=True,  # mandatory for hf models
#     max_new_tokens=512,
#     top_k=50,
#     top_p=0.95,
#     temperature=0.7,
#     # enforce_eager=True
# )


# def generate_text(prompt_text):

#     pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

#     # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
#     messages = [
#         {
#             "role": "system",
#             "content": "你是一個人工智慧助理以及台灣大學校園導覽員",
#         },
#         {"role": "user", "content": prompt_text},
#     ]
#     prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     print(f'Prompt:{prompt}')
#     outputs = llm(prompt)

#     # print(outputs[0]["generated_text"])
#     return outputs

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_id = "MediaTek-Research/Breeze-7B-Instruct-v0_1"
# model_id = "yentinglin/Taiwan-LLM-13B-v2.0-chat"
model_id = "taide/TAIDE-LX-7B-Chat"


# llm = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
    # torch_dtype=torch.bfloat16,
#     # attn_implementation="flash_attention_2" # optional
# ).cuda()
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    # torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    device_map="auto",
    # attn_implementation="flash_attention_2" # optional
)
tokenizer = AutoTokenizer.from_pretrained(model_id)


def generate_text(prompt_text):

    messages = [
        {
            "role": "system",
            "content": "你是一個人工智慧助理以及台灣大學校園導覽員",
        },
        {"role": "user", "content": prompt_text},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    # print(f'Prompt:{prompt}')

    inputs = tokenizer(prompt, return_tensors="pt").to(0)

    outputs = llm.generate(**inputs, max_new_tokens=512)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever

embeddings = HuggingFaceBgeEmbeddings(model_name = "BAAI/bge-large-zh-v1.5")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
# relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.55)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter]
)

db = FAISS.load_local('embeddings/all_bge_large_chatgpt', embeddings)
retriever = db.as_retriever(search_kwargs={"k": 5})
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)

In [ ]:
from tqdm import tqdm
import time

responses, retrv_urls, retrv_origin_docs, retrv_expand_docs = [], [], [], []

for ii, query in enumerate(tqdm(testset['Question'])):
    # print(f'Question: {query}')
    retrieve_docs = compression_retriever.get_relevant_documents(query)
    retrv_origin_docs.append([doc.page_content for doc in retrieve_docs])
    url_set = set([doc.metadata['url'] for doc in retrieve_docs])
    retrv_urls.append(url_set)

    contents = {}
    for url in url_set:
        for idx, u in enumerate(web_page['url']):
            if url==u:
                contents[url] = web_page['content'][idx]

    paragraphs = []
    for i in range(len(retrieve_docs)):
        snippet = retrieve_docs[i].page_content
        str_idx = contents[retrieve_docs[i].metadata['url']].find(snippet)
        if str_idx==-1:
            paragraphs.append(snippet)
        else:
            paragraphs.append(contents[retrieve_docs[i].metadata['url']][str_idx:str_idx+128])
    retrv_expand_docs.append(paragraphs)

    paragraph = [f'\n文檔{i+1}:'+paragraphs[i]+'' for i in range(len(paragraphs))]
# 回答時不要列出參考資料且
    prompt = '以下是參考資料，請忽略不相關的文件，回答盡量簡短精要，切勿重複輸出一樣文句子:{}\n請問:{}'.format(','.join(paragraph), query)
    # prompt = f'你是人工智慧助理，輸出繁體中文，你要對用戶的問題提供警慎且正確、安全、禮貌且簡潔精確的回答，以下是用戶和人工智能助理之間的對話。\nUSER: {prompt_template} \nASSISTANT:'
    # prompt = prompt_template

    # print('Prompt:')
    # print(prompt)

    # try:
    response = generate_text(prompt)
    # except:
        # time.sleep(5)
        # response = generate_text(prompt)['choices'][0]['message']['content']

    # response = generate_text(prompt)
    print("Response:")
    print(response)

    print('Ground Truth:')
    print(testset['Answer'][ii])
    print('\n*************************************************')

    responses.append(response)

In [ ]:
responses,  retrv_urls, retrv_origin_docs, retrv_expand_docs
testset[f'Answer_{model_id.split("/")[-1]}_RAG']=responses
testset['Retrieve_Expand_Content']=retrv_expand_docs
testset['Retrieve_Snippet_Content']=retrv_origin_docs
testset['Retreive_Urls']=retrv_urls
testset

In [ ]:
# testset.to_csv(f'evaluation/testset_{model_id.split("/")[1]}_RAG.csv',index=False)
testset.to_csv(f'evaluation/testset_{model_id.split("/")[-1]}_RAG.csv',index=False)